## solving QP primal

In [ ]:
using Random
using Ipopt
using MathOptInterface
using Dualization
using OSQP

const MOI = MathOptInterface;

In [ ]:
n = 20 # variable dimension
m = 15 # no of inequality constraints
p = 10; # no of equality constraints

## create a non-trivial QP problem

In [ ]:
x̂ = rand(n)
Q = rand(n, n)
Q = Q'*Q # ensure PSD
q = rand(n)
G = rand(m, n)
h = G*x̂ + rand(m)
A = rand(p, n)
b = A*x̂;

In [ ]:
model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
x = MOI.add_variables(model, n);

In [ ]:
# define objective

quad_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:n
    for j in i:n # indexes (i,j), (j,i) will be mirrored. specify only one kind
        push!(quad_terms, MOI.ScalarQuadraticTerm(Q[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x),quad_terms,0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

In [ ]:
# add constraints
for i in 1:m
    MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[i,:], x), -h[i]),MOI.LessThan(0.))
end

for i in 1:p
    MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i,:], x), -b[i]),MOI.EqualTo(0.))
end

In [ ]:
MOI.optimize!(model)

In [ ]:
@assert MOI.get(model, MOI.TerminationStatus()) in [MOI.LOCALLY_SOLVED, MOI.OPTIMAL]

In [ ]:
x̄ = MOI.get(model, MOI.VariablePrimal(), x);

In [ ]:
# objective value (predicted vs actual) sanity check
@assert 0.5*x̄'*Q*x̄ + q'*x̄  <= 0.5*x̂'*Q*x̂ + q'*x̂   

## find and solve dual problem 

| primal | dual |
|--------|------|
$$\text{min } \frac{1}{2}x^TQx + q^Tx$$  | $$\text{max } -\frac{1}{2}y^TQ^{-1}y - u^Th - v^Tb$$
$$\text{s.t.  }Gx <= h$$ | $$\text{s.t.  } u \geq 0, u \in R^m, v \in R^n$$
   $$Ax = b$$  |    $$y = q + G^Tu + A^Tv$$
  
- Each primal variable becomes a dual constraint
- Each primal constraint becomes a dual variable

In [ ]:
joint_object    = dualize(model)
dual_model_like = joint_object.dual_model # this is MOI.ModelLike, not an MOI.AbstractOptimizer; can't call optimizer on it
primal_dual_map = joint_object.primal_dual_map;

In [ ]:
# copy the dual model objective, constraints, and variables to an optimizer
dual_model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
MOI.copy_to(dual_model, dual_model_like)

# solve dual
MOI.optimize!(dual_model);

In [ ]:
# check if strong duality holds
@assert abs(MOI.get(model, MOI.ObjectiveValue()) - MOI.get(dual_model, MOI.ObjectiveValue())) <= 1e-2